### Importa módulos necessários

In [75]:
from pyspark.sql import SparkSession, Row, Window
import pyspark.sql.types as T
import pyspark.sql.functions as F

### Inicia sessão Spark

In [3]:
spark = (
    SparkSession.builder.master("local[*]")
    .appName("exploratory-data-analysis")
    .config("spark.ui.port", "4040")
    .config("spark.ui.showConsoleProgress", "True")
    .getOrCreate()
)

### Carrega tabela da camada bronze

In [63]:
bronze_path = "/home/jovyan/data/bronze/salaries.parquet"
df = spark.read.parquet(bronze_path)

In [64]:
df.show(truncate=False)

+----------------+---------------+-------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|experience_level|employment_type|job_title          |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|work_year|
+----------------+---------------+-------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|MI              |FT             |Data Manager       |117400|USD            |117400       |US                |0           |US              |M           |2024     |
|MI              |FT             |Data Manager       |62620 |USD            |62620        |US                |0           |US              |M           |2024     |
|SE              |FT             |Data Manager       |131200|USD            |131200       |US                |100         |US              |M           |2024     |
|SE             

In [65]:
df.describe().show(truncate=False)

+-------+----------------+---------------+----------------+------------------+---------------+------------------+------------------+-----------------+----------------+------------+-------------------+
|summary|experience_level|employment_type|job_title       |salary            |salary_currency|salary_in_usd     |employee_residence|remote_ratio     |company_location|company_size|work_year          |
+-------+----------------+---------------+----------------+------------------+---------------+------------------+------------------+-----------------+----------------+------------+-------------------+
|count  |64934           |64934          |64934           |64934             |64934          |64934             |64934             |64934            |64934           |64934       |64934              |
|mean   |NULL            |NULL           |NULL            |163455.40835309698|NULL           |158714.31233252224|NULL              |21.89068900729972|NULL            |NULL        |2023.80296916869

In [69]:
df.count()

64934

In [70]:
df.distinct().count()

30355

In [ ]:
df.groupBy(df.columns).agg(F.count("*").alias("count")).sort(
    "count", ascending=False
).show(truncate=False)

+----------------+---------------+---------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+-----+
|experience_level|employment_type|job_title                  |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|work_year|count|
+----------------+---------------+---------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+-----+
|SE              |FT             |Data Scientist             |160000|USD            |160000       |US                |100         |US              |M           |2024     |181  |
|SE              |FT             |Data Scientist             |110000|USD            |110000       |US                |100         |US              |M           |2024     |168  |
|SE              |FT             |Machine Learning Researcher|110000|USD            |110000       |US         

In [83]:
df_id = df.withColumn("id", F.monotonically_increasing_id()).select("id", *df.columns)

In [84]:
df_id.show(truncate=False)

+---+----------------+---------------+-------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|id |experience_level|employment_type|job_title          |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|work_year|
+---+----------------+---------------+-------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|0  |MI              |FT             |Data Manager       |117400|USD            |117400       |US                |0           |US              |M           |2024     |
|1  |MI              |FT             |Data Manager       |62620 |USD            |62620        |US                |0           |US              |M           |2024     |
|2  |SE              |FT             |Data Manager       |131200|USD            |131200       |US                |100         |US              |M           |202

In [85]:
df_id.distinct().count()

64934

In [86]:
def count_categories(df, column):
    return df.groupBy(column).agg(F.count("*").alias("count"))

In [88]:
for column in df_id.columns:
    print(f"Analisando coluna {column} ...")
    count_categories(df_id, column).sort("count", ascending=False).show(truncate=False)

Analisando coluna id ...
+----+-----+
|id  |count|
+----+-----+
|26  |1    |
|29  |1    |
|474 |1    |
|964 |1    |
|1677|1    |
|1697|1    |
|1806|1    |
|1950|1    |
|2040|1    |
|2214|1    |
|2250|1    |
|2453|1    |
|2509|1    |
|2529|1    |
|2927|1    |
|3091|1    |
|3506|1    |
|3764|1    |
|4590|1    |
|4823|1    |
+----+-----+
only showing top 20 rows

Analisando coluna experience_level ...
+----------------+-----+
|experience_level|count|
+----------------+-----+
|SE              |38223|
|MI              |19381|
|EN              |5962 |
|EX              |1368 |
+----------------+-----+

Analisando coluna employment_type ...
+---------------+-----+
|employment_type|count|
+---------------+-----+
|FT             |64653|
|PT             |145  |
|CT             |120  |
|FL             |16   |
+---------------+-----+

Analisando coluna job_title ...
+------------------------------+-----+
|job_title                     |count|
+------------------------------+-----+
|Data Scientist  

In [89]:
df.filter(F.col("salary_currency") == "BRL").show(truncate=False)

+----------------+---------------+-----------------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|experience_level|employment_type|job_title                          |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|work_year|
+----------------+---------------+-----------------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+
|SE              |FT             |Software Developer                 |368200|BRL            |75142        |BR                |0           |BR              |M           |2024     |
|SE              |FT             |Software Developer                 |216200|BRL            |44122        |BR                |0           |BR              |M           |2024     |
|SE              |FT             |Software Engineer                  |443000|BRL            |90408  

In [90]:
df.select("job_title").distinct().count()

267

In [91]:
df.groupBy("job_title").agg(F.count("*").alias("count")).sort(
    "count", ascending=False
).show(100, truncate=False)

+-----------------------------------------+-----+
|job_title                                |count|
+-----------------------------------------+-----+
|Data Scientist                           |10570|
|Data Engineer                            |8707 |
|Software Engineer                        |6767 |
|Data Analyst                             |6581 |
|Machine Learning Engineer                |5433 |
|Engineer                                 |3282 |
|Research Scientist                       |2153 |
|Manager                                  |2006 |
|Applied Scientist                        |1478 |
|Data Architect                           |1159 |
|Analyst                                  |1087 |
|Analytics Engineer                       |1032 |
|Research Engineer                        |958  |
|Product Manager                          |856  |
|Associate                                |768  |
|AI Engineer                              |547  |
|Business Intelligence Engineer           |542  |


In [112]:
df_job_focus = df_id.withColumn(
    "job_focus",
    F.when(F.contains(F.col("job_title"), F.lit("Engineer")), "Engineer")
    .when(F.contains(F.col("job_title"), F.lit("Scien")), "Scientist")
    .when(F.contains(F.col("job_title"), F.lit("Research")), "Scientist")
    .when(F.contains(F.col("job_title"), F.lit("Analy")), "Analyst")
    .when(F.contains(F.col("job_title"), F.lit("Architect")), "Architect")
    .when(F.contains(F.col("job_title"), F.lit("Model")), "Architect")
    .when(
        F.contains(F.col("job_title"), F.lit("Business Intelligence")),
        "Business Intelligence",
    )
    .when(F.contains(F.col("job_title"), F.lit("BI")), "Business Intelligence")
    .when(F.contains(F.col("job_title"), F.lit("Visual")), "Business Intelligence")
    .when(F.contains(F.col("job_title"), F.lit("Manager")), "Manager")
    .otherwise("Others"),
)

In [113]:
df_job_focus.groupBy("job_focus").agg(F.count("*").alias("count")).show(truncate=False)

+---------------------+-----+
|job_focus            |count|
+---------------------+-----+
|Scientist            |15665|
|Architect            |1859 |
|Analyst              |9337 |
|Others               |3372 |
|Engineer             |30172|
|Manager              |3647 |
|Business Intelligence|882  |
+---------------------+-----+



In [114]:
df_job_focus.filter(F.col("job_focus") == "Others").distinct().count()

3372

In [115]:
df_job_focus.filter(F.col("job_focus") == "Others").groupBy("job_title").agg(
    F.count("*").alias("count")
).sort("count", ascending=False).show(100, truncate=False)

+-----------------------------------------+-----+
|job_title                                |count|
+-----------------------------------------+-----+
|Associate                                |768  |
|Consultant                               |358  |
|Data Specialist                          |344  |
|Software Developer                       |272  |
|Head of Data                             |148  |
|Developer                                |100  |
|Data Lead                                |90   |
|AI Developer                             |81   |
|Data Management Specialist               |71   |
|Data Developer                           |66   |
|Data Product Owner                       |54   |
|Statistical Programmer                   |54   |
|Data Strategist                          |52   |
|Technical Lead                           |50   |
|AI Specialist                            |48   |
|Data Integration Specialist              |41   |
|Principal Application Delivery Consultant|40   |


### Corrigir o salário em dólares pela inflação americana anual

[Referência](https://www.macrotrends.net/global-metrics/countries/USA/united-states/inflation-rate-cpi)

In [ ]:
inflation = spark.createDataFrame(
    [
        Row(year=2020, rate=0.0123),
        Row(year=2021, rate=0.0470),
        Row(year=2022, rate=0.0800),
        Row(year=2023, rate=0.0412),
        Row(year=2024, rate=0.0),
    ]
)

In [126]:
inflation.show(truncate=False)

+----+------+
|year|rate  |
+----+------+
|2020|0.0123|
|2021|0.047 |
|2022|0.08  |
|2023|0.0412|
|2024|0.0   |
+----+------+



In [127]:
cumulative_inflation = (
    inflation.select("year")
    .alias("a")
    .join(inflation.alias("b"), F.col("b.year") >= F.col("a.year"), how="left")
    .select("a.year", "rate")
    .groupBy("year")
    .agg(F.sum(F.col("rate")).alias("cumulative_rate"))
)

In [ ]:
cumulative_inflation.sort("year").show(truncate=False)

+----+-------------------+
|year|cumulative_rate    |
+----+-------------------+
|2020|0.1805             |
|2021|0.16820000000000002|
|2022|0.1212             |
|2023|0.0412             |
|2024|0.0                |
+----+-------------------+



In [ ]:
df_update = (
    df_job_focus.join(
        cumulative_inflation, df.work_year == cumulative_inflation.year, how="left"
    )
    .withColumn(
        "updated_salary",
        F.round(
            F.col("salary_in_usd") + F.col("salary_in_usd") * F.col("cumulative_rate"),
            2,
        ),
    )
    .sort("work_year")
)

In [131]:
df_update.show(truncate=False)

+-----------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+---------+----+---------------+--------------+
|id         |experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|work_year|job_focus|year|cumulative_rate|updated_salary|
+-----------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------+---------+----+---------------+--------------+
|17179869184|EN              |FT             |Azure Data Engineer      |100000|USD            |100000       |MU                |0           |MU              |S           |2020     |Engineer |2020|0.1805         |118050.0      |
|17179869204|EN              |FT             |Machine Learning Engineer|250000|USD      

In [132]:
def evaluate_response_variable(df, column):
    return df.groupBy(column).agg(
        F.count("*").alias("count"),
        F.mean("updated_salary").alias("mean_updated_salary"),
    )

In [133]:
for column in df_update.columns:
    evaluate_response_variable(df_update, column).sort(column).show(truncate=False)

+---+-----+-------------------+
|id |count|mean_updated_salary|
+---+-----+-------------------+
|0  |1    |117400.0           |
|1  |1    |62620.0            |
|2  |1    |131200.0           |
|3  |1    |95300.0            |
|4  |1    |68800.0            |
|5  |1    |48000.0            |
|6  |1    |264200.0           |
|7  |1    |143100.0           |
|8  |1    |208000.0           |
|9  |1    |141340.0           |
|10 |1    |200000.0           |
|11 |1    |136000.0           |
|12 |1    |329600.0           |
|13 |1    |190700.0           |
|14 |1    |360200.0           |
|15 |1    |208300.0           |
|16 |1    |264200.0           |
|17 |1    |143100.0           |
|18 |1    |312200.0           |
|19 |1    |175800.0           |
+---+-----+-------------------+
only showing top 20 rows

+----------------+-----+-------------------+
|experience_level|count|mean_updated_salary|
+----------------+-----+-------------------+
|EN              |5962 |103807.70805769876 |
|EX              |1368 |20

### Treinando modelo de regressão

In [141]:
from pyspark.mllib.tree import RandomForest
from pyspark.ml.feature import VectorIndexer, VectorAssembler

In [144]:
assembler = VectorAssembler(
    inputCols=[
        "experience_level",
        "employment_type",
        "employee_residence",
        "remote_ratio",
        "company_size",
    ],
    outputCol="features",
)

In [ ]:
output = indexer.transform(df_update)

IllegalArgumentException: Data type string of column experience_level is not supported.
Data type string of column employment_type is not supported.
Data type string of column employee_residence is not supported.
Data type string of column company_size is not supported.

In [136]:
model = RandomForest.trainRegressor(df_update, {}, 10)

AssertionError: the data should be RDD of LabeledPoint